In [1]:
import os 

In [2]:
%pwd

'd:\\Machine-learning-project-with-Mlflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Machine-learning-project-with-Mlflow'

In [6]:
import pandas as pd 

In [10]:
df = pd.read_csv('artifacts/data_ingestion/winequality-red.csv').head()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5 non-null      float64
 1   volatile acidity      5 non-null      float64
 2   citric acid           5 non-null      float64
 3   residual sugar        5 non-null      float64
 4   chlorides             5 non-null      float64
 5   free sulfur dioxide   5 non-null      float64
 6   total sulfur dioxide  5 non-null      float64
 7   density               5 non-null      float64
 8   pH                    5 non-null      float64
 9   sulphates             5 non-null      float64
 10  alcohol               5 non-null      float64
 11  quality               5 non-null      int64  
dtypes: float64(11), int64(1)
memory usage: 612.0 bytes


In [12]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [25]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,    
        schema_file_path: Path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation 
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=Path(config.root_dir),
            STATUS_FILE=Path(config.STATUS_FILE),  # Changed back to uppercase
            unzip_data_dir=Path(config.unzip_data_dir),
            all_schema=schema
    )

        return data_validation_config

In [16]:
import os
from mlProject import logger


In [26]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            
            # Read the data
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            
            # Get schema columns
            all_schema = self.config.all_schema.keys()
            
            # Check if all columns match
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    break
            else:
                validation_status = True
            
            # Write validation status to file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")
            
            return validation_status
        
        except Exception as e:
            raise e

In [27]:
from mlProject.utils.common import read_yaml
from pathlib import Path

schema = read_yaml(Path("config/schema.yaml"))
print(schema)
print("\nKeys in schema:", schema.keys())

[2025-12-25 21:51:54,350 : INFO : common : YAML file 'config\schema.yaml' read successfully.]
{'COLUMNS': {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}, 'TARGET_COLUMN': {'name': 'quality'}}

Keys in schema: dict_keys(['COLUMNS', 'TARGET_COLUMN'])


In [28]:
from mlProject.utils.common import read_yaml
from pathlib import Path

schema = read_yaml(Path("config/schema.yaml"))
print("COLUMNS found:", "COLUMNS" in schema)
print("\nColumns:", schema.COLUMNS)

[2025-12-25 21:51:57,929 : INFO : common : YAML file 'config\schema.yaml' read successfully.]
COLUMNS found: True

Columns: {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}


In [29]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-12-25 21:52:01,178 : INFO : common : YAML file 'config\config.yaml' read successfully.]
[2025-12-25 21:52:01,183 : INFO : common : YAML file 'params.yaml' read successfully.]
[2025-12-25 21:52:01,193 : INFO : common : YAML file 'config\schema.yaml' read successfully.]
[2025-12-25 21:52:01,197 : INFO : common : Created directory at: artifacts_root]
[2025-12-25 21:52:01,202 : INFO : common : Created directory at: artifacts/data_validation]
